In [1]:
import io
import os
import sys
import requests
from collections import OrderedDict 
import math
import random
import numpy as np
import paddle
import paddle.fluid as fluid

from paddle.fluid.dygraph.nn import Embedding

In [2]:
#读取数据
def load_data():
    with open("./article_data.txt", "r") as f:
        corpus = f.readlines()
    f.close()

    return corpus

corpus = load_data()

print(corpus[:50])

['林子大了什么鸟都有太珍稀了\n', '玻璃瓶里居然装了一幅山水画太美了\n', '一组用微信表情组成的话有点难度能猜出的人几乎为零\n', '大大大大大大大大大大大大大实在太大了\n', '天呐她太漂亮了实在太漂亮了忍不住发给你看看\n', '最美长寿花看到的人有福了\n', '肉ྂ体照片罕见实在太罕见了\n', '世界上最好的长寿药竟然不花一分钱\n', '一组照片缓解眼睛疲劳赶紧收藏\n', '令人销魂的照片别不好意思看\n', '手心里的宝贝太可爱了\n', '十句话送给晚上睡不着觉的人精辟\n', '惊滟全果照看了流口水\n', '牛牛牛让人目瞪口呆的照片\n', '百万都买不到这张表收藏\n', '奶奶为孙子编织的毛线大全太可爱了\n', '会动照片绝了绝了绝了绝了\n', '太牛了高手在民间开开眼界吧\n', '月色美人图一辈子看次值了\n', '再见了朝鲜姑娘\n', '七彩幸福鸟七彩幸福鸟七彩幸福鸟七彩幸福鸟\n', '中国女兵太美太震撼了\n', '神一样的抓拍刚看到第二张我就笑疯了\n', '个民族个美女很难找齐太美了\n', '餐厅里的罕见照片太厉害了\n', '她是最美妲己拍过三级片结婚十年不生孩子丈夫仍然捧在手心岁美如少女\n', '最美的花最好的祝福送给你\n', '罕见生子年画最后一张太珍贵了\n', '全国市市花太美了\n', '太美了美到心坎上\n', '罕见罕见太罕见了从没见过这样的鸟\n', '美美美美美看到的人有福气了\n', '美丽花鸟美艳极了\n', '一组看哭了无数中国人的老照片\n', '罕见珍鸟从未见过太太太太漂亮了\n', '吉祥长寿相册送给你\n', '肉上开花难得一见赶紧珍藏\n', '罕见至极七彩菊花美到窒息\n', '荷花荷花荷花荷花荷花荷花荷花荷花荷花荷花荷花\n', '鸟鸟鸟鸟鸟鸟鸟鸟鸟鸟鸟鸟鸟鸟鸟鸟\n', '刚刚抓拍的真是太太太绝了\n', '佛佛佛谁看谁有福\n', '全世界的钱你见过没真是大开眼界\n', '世界奇观景你绝对不敢相信这是真的\n', '外国人眼里最美的中国女人居然是年过的她\n', '我的家庭相册\n', '绝版相册\n', '雪中玫瑰美到心醉\n', '我的相册四\n', '承德避暑山庄全景美不胜收\n']


In [3]:
# 对语料进行预处理（分词）
def data_preprocess(corpus):
    import jieba
    import jieba.analyse
    import jieba.posseg as pseg
    import re

    filtered_words_list = []

    punc = ' ~`!#$%^&*()_+-=|\;":/.,?><~·！\n@#￥%……&*（）——+-=“”：’；、。，？{}'

    for sentence in corpus:
        sentence = re.sub(r"[%s]+" % punc, "", sentence)
        seg_list = pseg.lcut(sentence)
        for seg in seg_list:
            if seg.flag.startswith('n'):
                filtered_words_list.append(seg.word)
    
    return filtered_words_list

corpus = data_preprocess(corpus)
print(corpus[:50])

Building prefix dict from the default dictionary ...
2020-05-15 11:48:33,823-DEBUG: Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
2020-05-15 11:48:34,941-DEBUG: Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.199 seconds.
2020-05-15 11:48:35,024-DEBUG: Loading model cost 1.199 seconds.
Prefix dict has been built successfully.
2020-05-15 11:48:35,026-DEBUG: Prefix dict has been built successfully.


['林子', '鸟', '玻璃瓶', '山水画', '表情', '有点', '人', '太漂亮', '太漂亮', '长寿', '人', '肉', '体', '照片', '世界', '长寿', '药', '一分钱', '照片', '眼睛', '令人', '销魂', '照片', '手', '宝贝', '人', '全果', '流口水', '牛牛牛', '人', '照片', '奶奶', '孙子', '编织', '毛线', '照片', '太牛', '高手', '民间', '开眼界', '月色', '美人图', '朝鲜', '姑娘', '七彩', '七彩', '七彩', '七彩', '中国', '女兵']


In [4]:
#构造词典，统计每个词的频率，并根据频率将每个词转换为一个整数id
def build_dict(corpus):
    #首先统计每个不同词的频率（出现的次数），使用一个词典记录
    word_freq_dict = dict()
    for word in corpus:
        if word not in word_freq_dict:
            word_freq_dict[word] = 0
        word_freq_dict[word] += 1

    #将这个词典中的词，按照出现次数排序，出现次数越高，排序越靠前
    #一般来说，出现频率高的高频词往往是：I，the，you这种代词，而出现频率低的词，往往是一些名词，如：nlp
    word_freq_dict = sorted(word_freq_dict.items(), key = lambda x:x[1], reverse = True)
    
    #构造3个不同的词典，分别存储，
    #每个词到id的映射关系：word2id_dict
    #每个id出现的频率：word2id_freq
    #每个id到词典映射关系：id2word_dict
    word2id_dict = dict()
    word2id_freq = dict()
    id2word_dict = dict()

    #按照频率，从高到低，开始遍历每个单词，并为这个单词构造一个独一无二的id
    for word, freq in word_freq_dict:
        curr_id = len(word2id_dict)
        word2id_dict[word] = curr_id
        word2id_freq[word2id_dict[word]] = freq
        id2word_dict[curr_id] = word

    return word2id_freq, word2id_dict, id2word_dict

word2id_freq, word2id_dict, id2word_dict = build_dict(corpus)
vocab_size = len(word2id_freq)
print("there are totoally %d different words in the corpus" % vocab_size)
for _, (word, word_id) in zip(range(50), word2id_dict.items()):
    print("word %s, its id %d, its word freq %d" % (word, word_id, word2id_freq[word_id]))

there are totoally 17719 different words in the corpus
word 人, its id 0, its word freq 8084
word 中国, its id 1, its word freq 3110
word 朋友, its id 2, its word freq 2247
word 女人, its id 3, its word freq 1504
word 广场, its id 4, its word freq 1257
word 经典, its id 5, its word freq 1244
word 视频, its id 6, its word freq 1137
word 男人, its id 7, its word freq 972
word 祝福, its id 8, its word freq 966
word 人生, its id 9, its word freq 965
word 世界, its id 10, its word freq 948
word 美女, its id 11, its word freq 908
word 战友, its id 12, its word freq 902
word 情歌, its id 13, its word freq 865
word 歌, its id 14, its word freq 844
word 群友, its id 15, its word freq 841
word 照片, its id 16, its word freq 828
word 农村, its id 17, its word freq 820
word 大家, its id 18, its word freq 791
word 孩子, its id 19, its word freq 790
word 儿子, its id 20, its word freq 765
word 小品, its id 21, its word freq 759
word 老婆, its id 22, its word freq 719
word 钱, its id 23, its word freq 574
word 心, its id 24, its word freq 567
wo

In [5]:
#把语料转换为id序列
def convert_corpus_to_id(corpus, word2id_dict):
    #使用一个循环，将语料中的每个词替换成对应的id，以便于神经网络进行处理
    corpus = [word2id_dict[word] for word in corpus]
    return corpus

corpus = convert_corpus_to_id(corpus, word2id_dict)
print("%d tokens in the corpus" % len(corpus))
print(corpus[:50])

174193 tokens in the corpus
[3292, 351, 6707, 1714, 275, 755, 0, 68, 68, 51, 0, 235, 5289, 16, 10, 51, 417, 670, 16, 280, 115, 2449, 16, 191, 821, 0, 9283, 1144, 1715, 0, 16, 199, 236, 4384, 4385, 16, 133, 170, 173, 1013, 5290, 2259, 822, 72, 683, 683, 683, 683, 1, 136]


In [6]:
#使用二次采样算法（subsampling）处理语料，强化训练效果
def subsampling(corpus, word2id_freq):
    
    #这个discard函数决定了一个词会不会被替换，这个函数是具有随机性的，每次调用结果不同
    #如果一个词的频率很大，那么它被遗弃的概率就很大
    def discard(word_id):
        return random.uniform(0, 1) < 1 - math.sqrt(
            1e-4 / word2id_freq[word_id] * len(corpus))

    corpus = [word for word in corpus if not discard(word)]
    return corpus

corpus = subsampling(corpus, word2id_freq)
print("%d tokens in the corpus" % len(corpus))
print(corpus[:50])

93881 tokens in the corpus
[3292, 351, 6707, 1714, 68, 235, 5289, 417, 670, 2449, 821, 9283, 1144, 1715, 199, 4384, 4385, 1013, 5290, 2259, 683, 683, 683, 489, 3755, 16, 9284, 2959, 656, 8, 2450, 1540, 4386, 3293, 232, 520, 9285, 128, 128, 119, 51, 683, 981, 551, 551, 551, 551, 551, 551, 551]


In [7]:
#构造数据，准备模型训练
#max_window_size代表了最大的window_size的大小，程序会根据max_window_size从左到右扫描整个语料
#negative_sample_num代表了对于每个正样本，需要随机采样多少负样本用于训练，
#一般来说，negative_sample_num的值越大，训练效果越稳定，但是训练速度越慢。 
def build_data(corpus, word2id_dict, word2id_freq, max_window_size = 1, negative_sample_num = 10):
    
    #使用一个list存储处理好的数据
    dataset = []

    #从左到右，开始枚举每个中心点的位置
    for center_word_idx in range(len(corpus)):
        #以max_window_size为上限，随机采样一个window_size，这样会使得训练更加稳定
        window_size = random.randint(1, max_window_size)
        #当前的中心词就是center_word_idx所指向的词
        center_word = corpus[center_word_idx]

        #以当前中心词为中心，左右两侧在window_size内的词都可以看成是正样本
        positive_word_range = (max(0, center_word_idx - window_size), min(len(corpus) - 1, center_word_idx + window_size))
        positive_word_candidates = [corpus[idx] for idx in range(positive_word_range[0], positive_word_range[1]+1) if idx != center_word_idx]

        #对于每个正样本来说，随机采样negative_sample_num个负样本，用于训练
        for positive_word in positive_word_candidates:
            #首先把（中心词，正样本，label=1）的三元组数据放入dataset中，
            #这里label=1表示这个样本是个正样本
            dataset.append((center_word, positive_word, 1))

            #开始负采样
            i = 0
            while i < negative_sample_num:
                negative_word_candidate = random.randint(0, vocab_size-1)

                if negative_word_candidate not in positive_word_candidates:
                    #把（中心词，正样本，label=0）的三元组数据放入dataset中，
                    #这里label=0表示这个样本是个负样本
                    dataset.append((center_word, negative_word_candidate, 0))
                    i += 1
    
    return dataset

dataset = build_data(corpus, word2id_dict, word2id_freq)
for _, (center_word, target_word, label) in zip(range(50), dataset):
    print("center_word %s, target %s, label %d" % (id2word_dict[center_word],
                                                   id2word_dict[target_word], label))

center_word 林子, target 鸟, label 1
center_word 林子, target 苦心, label 0
center_word 林子, target 社员, label 0
center_word 林子, target 樱花雨, label 0
center_word 林子, target 别提, label 0
center_word 林子, target 嫂嫂, label 0
center_word 林子, target 开花, label 0
center_word 林子, target 耿莲凤, label 0
center_word 林子, target 讲座, label 0
center_word 林子, target 功名, label 0
center_word 林子, target 塘鱼, label 0
center_word 鸟, target 林子, label 1
center_word 鸟, target 妇科病, label 0
center_word 鸟, target 浪费, label 0
center_word 鸟, target 公司, label 0
center_word 鸟, target 肚儿, label 0
center_word 鸟, target 南太平洋, label 0
center_word 鸟, target 佛版, label 0
center_word 鸟, target 风华, label 0
center_word 鸟, target 江, label 0
center_word 鸟, target 牛太牛, label 0
center_word 鸟, target 技师, label 0
center_word 鸟, target 玻璃瓶, label 1
center_word 鸟, target 求救信号, label 0
center_word 鸟, target 价格便宜, label 0
center_word 鸟, target 长寿, label 0
center_word 鸟, target 炒青菜, label 0
center_word 鸟, target 山竹, label 0
center_word 鸟, target 枪式, l

In [8]:
#构造mini-batch，准备对模型进行训练
#将不同类型的数据放到不同的tensor里，便于神经网络进行处理
#并通过numpy的array函数，构造出不同的tensor来，并把这些tensor送入神经网络中进行训练
def build_batch(dataset, batch_size, epoch_num):
    
    #center_word_batch缓存batch_size个中心词
    center_word_batch = []
    #target_word_batch缓存batch_size个目标词（可以是正样本或者负样本）
    target_word_batch = []
    #label_batch缓存了batch_size个0或1的标签，用于模型训练
    label_batch = []

    for epoch in range(epoch_num):
        #每次开启一个新epoch之前，都对数据进行一次随机打乱，提高训练效果
        random.shuffle(dataset)
        
        for center_word, target_word, label in dataset:
            #遍历dataset中的每个样本，并将这些数据送到不同的tensor里
            center_word_batch.append([center_word])
            target_word_batch.append([target_word])
            label_batch.append(label)

            #当样本积攒到一个batch_size后，把数据都返回回来
            #在这里使用numpy的array函数把list封装成tensor
            #并使用python的迭代器机制，将数据yield出来
            #使用迭代器的好处是可以节省内存
            if len(center_word_batch) == batch_size:
                yield np.array(center_word_batch).astype("int64"), \
                    np.array(target_word_batch).astype("int64"), \
                    np.array(label_batch).astype("float32")
                center_word_batch = []
                target_word_batch = []
                label_batch = []

    if len(center_word_batch) > 0:
        yield np.array(center_word_batch).astype("int64"), \
            np.array(target_word_batch).astype("int64"), \
            np.array(label_batch).astype("float32")

for _, batch in zip(range(10), build_batch(dataset, 128, 3)):
    print(batch)

In [9]:
#定义skip-gram训练网络结构
#这里使用的是paddlepaddle的1.7.0版本
#一般来说，在使用fluid训练的时候，需要通过一个类来定义网络结构，这个类继承了fluid.dygraph.Layer
class SkipGram(fluid.dygraph.Layer):
    def __init__(self, vocab_size, embedding_size, init_scale=0.1):
        #vocab_size定义了这个skipgram这个模型的词表大小
        #embedding_size定义了词向量的维度是多少
        #init_scale定义了词向量初始化的范围，一般来说，比较小的初始化范围有助于模型训练
        super(SkipGram, self).__init__()
        self.vocab_size = vocab_size
        self.embedding_size = embedding_size

        #使用paddle.fluid.dygraph提供的Embedding函数，构造一个词向量参数
        #这个参数的大小为：[self.vocab_size, self.embedding_size]
        #数据类型为：float32
        #这个参数的名称为：embedding_para
        #这个参数的初始化方式为在[-init_scale, init_scale]区间进行均匀采样
        self.embedding = Embedding(
            size=[self.vocab_size, self.embedding_size],
            dtype='float32',
            param_attr=fluid.ParamAttr(
                name='embedding_para',
                initializer=fluid.initializer.UniformInitializer(
                    low=-0.5/embedding_size, high=0.5/embedding_size)))

        #使用paddle.fluid.dygraph提供的Embedding函数，构造另外一个词向量参数
        #这个参数的大小为：[self.vocab_size, self.embedding_size]
        #数据类型为：float32
        #这个参数的名称为：embedding_para_out
        #这个参数的初始化方式为在[-init_scale, init_scale]区间进行均匀采样
        #跟上面不同的是，这个参数的名称跟上面不同，因此，
        #embedding_para_out和embedding_para虽然有相同的shape，但是权重不共享
        self.embedding_out = Embedding(
            size=[self.vocab_size, self.embedding_size],
            dtype='float32',
            param_attr=fluid.ParamAttr(
                name='embedding_out_para',
                initializer=fluid.initializer.UniformInitializer(
                    low=-0.5/embedding_size, high=0.5/embedding_size)))

    #定义网络的前向计算逻辑
    #center_words是一个tensor（mini-batch），表示中心词
    #target_words是一个tensor（mini-batch），表示目标词
    #label是一个tensor（mini-batch），表示这个词是正样本还是负样本（用0或1表示）
    #用于在训练中计算这个tensor中对应词的同义词，用于观察模型的训练效果
    def forward(self, center_words, target_words, label):
        #首先，通过embedding_para（self.embedding）参数，将mini-batch中的词转换为词向量
        #这里center_words和eval_words_emb查询的是一个相同的参数
        #而target_words_emb查询的是另一个参数
        center_words_emb = self.embedding(center_words)
        target_words_emb = self.embedding_out(target_words)

        #center_words_emb = [batch_size, embedding_size]
        #target_words_emb = [batch_size, embedding_size]
        #通过点乘的方式计算中心词到目标词的输出概率，并通过sigmoid函数估计这个词是正样本还是负样本的概率。
        word_sim = fluid.layers.elementwise_mul(center_words_emb, target_words_emb)
        word_sim = fluid.layers.reduce_sum(word_sim, dim = -1)
        word_sim = fluid.layers.reshape(word_sim, shape=[-1])
        pred = fluid.layers.sigmoid(word_sim)

        #通过估计的输出概率定义损失函数，注意使用的是sigmoid_cross_entropy_with_logits函数
        #将sigmoid计算和cross entropy合并成一步计算可以更好的优化，所以输入的是word_sim，而不是pred
        
        loss = fluid.layers.sigmoid_cross_entropy_with_logits(word_sim, label)
        loss = fluid.layers.reduce_mean(loss)

        #返回前向计算的结果，飞桨会通过backward函数自动计算出反向结果。
        return pred, loss

In [10]:
#开始训练，定义一些训练过程中需要使用的超参数
batch_size = 512
epoch_num = 3
embedding_size = 200
step = 0
learning_rate = 0.001

#定义一个使用word-embedding查询同义词的函数
#这个函数query_token是要查询的词，k表示要返回多少个最相似的词，embed是学习到的word-embedding参数
#通过计算不同词之间的cosine距离，来衡量词和词的相似度
#具体实现如下，x代表要查询词的Embedding，Embedding参数矩阵W代表所有词的Embedding
#两者计算Cos得出所有词对查询词的相似度得分向量，排序取top_k放入indices列表
def get_similar_tokens(query_token, k, embed):
    W = embed.numpy()
    x = W[word2id_dict[query_token]]
    cos = np.dot(W, x) / np.sqrt(np.sum(W * W, axis=1) * np.sum(x * x) + 1e-9)
    flat = cos.flatten()
    indices = np.argpartition(flat, -k)[-k:]
    indices = indices[np.argsort(-flat[indices])]
    for i in indices:
        print('for word %s, the similar word is %s' % (query_token, str(id2word_dict[i])))

#将模型放到GPU上训练（fluid.CUDAPlace(0)），如果需要指定CPU，则需要改为fluid.CPUPlace()
with fluid.dygraph.guard(fluid.CPUPlace()):
    #通过定义的SkipGram类，来构造一个Skip-gram模型网络
    skip_gram_model = SkipGram(vocab_size, embedding_size)
    #构造训练这个网络的优化器
    adam = fluid.optimizer.AdamOptimizer(learning_rate=learning_rate, parameter_list = skip_gram_model.parameters())

    #使用build_batch函数，以mini-batch为单位，遍历训练数据，并训练网络
    for center_words, target_words, label in build_batch(
        dataset, batch_size, epoch_num):
        #使用fluid.dygraph.to_variable函数，将一个numpy的tensor，转换为飞桨可计算的tensor
        center_words_var = fluid.dygraph.to_variable(center_words)
        target_words_var = fluid.dygraph.to_variable(target_words)
        label_var = fluid.dygraph.to_variable(label)

        #将转换后的tensor送入飞桨中，进行一次前向计算，并得到计算结果
        pred, loss = skip_gram_model(
            center_words_var, target_words_var, label_var)

        #通过backward函数，让程序自动完成反向计算
        loss.backward()
        #通过minimize函数，让程序根据loss，完成一步对参数的优化更新
        adam.minimize(loss)
        #使用clear_gradients函数清空模型中的梯度，以便于下一个mini-batch进行更新
        skip_gram_model.clear_gradients()

        #每经过100个mini-batch，打印一次当前的loss，看看loss是否在稳定下降
        step += 1
        if step % 100 == 0:
            print("step %d, loss %.3f" % (step, loss.numpy()[0]))

        #经过10000个mini-batch，打印一次模型对eval_words中的10个词计算的同义词
        #这里使用词和词之间的向量点积作为衡量相似度的方法
        #只打印了10个最相似的词
        if step % 1000 == 0:
            get_similar_tokens('病毒', 10, skip_gram_model.embedding.weight)
            get_similar_tokens('钟南山', 10, skip_gram_model.embedding.weight)
            get_similar_tokens('武汉', 10, skip_gram_model.embedding.weight)

step 100, loss 0.693
step 200, loss 0.692
step 300, loss 0.687
step 400, loss 0.655
step 500, loss 0.584
step 600, loss 0.473
step 700, loss 0.441
step 800, loss 0.347
step 900, loss 0.329
step 1000, loss 0.286
for word 病毒, the similar word is 病毒
for word 病毒, the similar word is 舞蹈
for word 病毒, the similar word is 流水
for word 病毒, the similar word is 信
for word 病毒, the similar word is 醉人
for word 病毒, the similar word is 长寿
for word 病毒, the similar word is 肺炎
for word 病毒, the similar word is 小女孩
for word 病毒, the similar word is 大气
for word 病毒, the similar word is 感人
for word 钟南山, the similar word is 钟南山
for word 钟南山, the similar word is 小心
for word 钟南山, the similar word is 母亲
for word 钟南山, the similar word is 妻子
for word 钟南山, the similar word is 大家
for word 钟南山, the similar word is 老公
for word 钟南山, the similar word is 文
for word 钟南山, the similar word is 情歌
for word 钟南山, the similar word is 祝你开心
for word 钟南山, the similar word is 图
for word 武汉, the similar word is 武汉
for word 武汉, the simil

In [24]:
get_similar_tokens('肺炎', 20, skip_gram_model.embedding.weight)

for word 肺炎, the similar word is 肺炎
for word 肺炎, the similar word is 疫苗
for word 肺炎, the similar word is 勤洗手
for word 肺炎, the similar word is 朱广权
for word 肺炎, the similar word is 阳性
for word 肺炎, the similar word is 流感
for word 肺炎, the similar word is 重大成果
for word 肺炎, the similar word is 最高人民检察院
for word 肺炎, the similar word is 病毒
for word 肺炎, the similar word is 全纪录
for word 肺炎, the similar word is 法医
for word 肺炎, the similar word is 钟南山
for word 肺炎, the similar word is 发明人
for word 肺炎, the similar word is 费用
for word 肺炎, the similar word is 德塞
for word 肺炎, the similar word is 二龙
for word 肺炎, the similar word is 粮价
for word 肺炎, the similar word is 新冠
for word 肺炎, the similar word is 中文
for word 肺炎, the similar word is 李兰娟
